In [1]:
import cirq
import numpy as np

In [2]:
#Create qutrit hadamard
w = np.exp(2 * np.pi * 1j / 3)

hUnitary = np.array([[1, 1, 1],
                         [1, w, w**2],
                         [1, w**2, w**4]]) / np.sqrt(3)

class QutritHadamard(cirq.SingleQubitGate):
    def _qid_shape_(self):
        return (3,)

    def _unitary_(self):
        return hUnitary

    def _circuit_diagram_info_(self, args):
        return '[H+]'

print(hUnitary)

[[ 0.57735027+0.j   0.57735027+0.j   0.57735027+0.j ]
 [ 0.57735027+0.j  -0.28867513+0.5j -0.28867513-0.5j]
 [ 0.57735027+0.j  -0.28867513-0.5j -0.28867513+0.5j]]


In [4]:
#qutrit inverse hadamard
#H^4 = I, so H^-1 = H^3

hInvUnitary = np.array([[1, 1, 1],
                         [1, w**2, w],
                         [1, w, w**2]]) / np.sqrt(3)

class QutritInvHadamard(cirq.SingleQubitGate):
    def _qid_shape_(self):
        return (3,)

    def _unitary_(self):
        return hInvUnitary

    def _circuit_diagram_info_(self, args):
        return '[H-]'

print(hInvUnitary)

[[ 0.57735027+0.j   0.57735027+0.j   0.57735027+0.j ]
 [ 0.57735027+0.j  -0.28867513-0.5j -0.28867513+0.5j]
 [ 0.57735027+0.j  -0.28867513+0.5j -0.28867513-0.5j]]


In [5]:
#Create CNOT gate for qutrit
cUnitary = np.zeros((9, 9))

#Indices to turn to 1
ons = [
    [0, 0],
    [1, 1],
    [2, 2],
    [3, 5],
    [4, 3],
    [5, 4],
    [6, 7],
    [7, 8],
    [8, 6]
]
for inds in ons:
    cUnitary[inds[0]][inds[1]] = 1
    
class QutritCnot(cirq.Gate):
    def __init__(self):
        super(QutritCnot, self)
    
    def _num_qubits_self(self):
        return 2
    
    def _qid_shape_(self):
        return (3,3)

    def _unitary_(self):
        return cUnitary

    def _circuit_diagram_info_(self, args):
        return 'O', 'X'
    
print(cUnitary)

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]]


In [6]:
#Create Inverse CNOT Gate, transpose of cnot
cInvUnitary = np.transpose(cUnitary)
class QutritInvCnot(cirq.Gate):
    def __init__(self):
        super(QutritInvCnot, self)
    
    def _num_qubits_self(self):
        return 2
    
    def _qid_shape_(self):
        return (3,3)

    def _unitary_(self):
        return cInvUnitary

    def _circuit_diagram_info_(self, args):
        return 'O', 'Y'
    
print(cInvUnitary)

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [7]:
class QutritPlusGate(cirq.SingleQubitGate):
    def _qid_shape_(self):
        return (3,)

    def _unitary_(self):
        return np.array([[0, 0, 1],
                         [1, 0, 0],
                         [0, 1, 0]])

    def _circuit_diagram_info_(self, args):
        return '[+1]'

In [8]:
class QutritMinusGate(cirq.SingleQubitGate):
    def _qid_shape_(self):
        return (3,)

    def _unitary_(self):
        return np.array([[0, 1, 0],
                         [0, 0, 1],
                         [1, 0, 0]])

    def _circuit_diagram_info_(self, args):
        return '[-1]'

In [9]:
class QutritZGate(cirq.SingleQubitGate):
    def _qid_shape_(self):
        return (3,)

    def _unitary_(self):
        return np.array([[1, 0, 0],
                         [0, w, 0],
                         [0, 0, w**2]])

    def _circuit_diagram_info_(self, args):
        return 'Z'

In [15]:
def seeState(circuit):
    s = cirq.Simulator()
    results = s.simulate(circuit)
    
    print("Circuit: ")
    print(circuit)
    print("\nThe state is:\n {}".format(results.dirac_notation()))

def entangle(q0, q1, circuit):
    circuit.append(QutritHadamard().on(q0))
    circuit.append(QutritCnot().on(q0, q1))
    
def aliceOperation(values, q0, circuit):
    for i in range(values[1]):
        circuit.append(QutritMinusGate().on(q0))
    
    for i in range(values[0]):
        circuit.append(QutritZGate().on(q0))
    
        
def bobOperation(q0, q1, circuit):
    circuit.append(QutritInvCnot().on(q0, q1))
    circuit.append(QutritInvHadamard().on(q0))

In [53]:
#Define 2 digit base 3 number to be transmitted
values = (2, 2)

#Entangle qutrits
q0 = cirq.LineQid(0, dimension=3)
q1 = cirq.LineQid(1, dimension=3)
circuit = cirq.Circuit()
entangle(q0, q1, circuit)
seeState(circuit)


Circuit: 
0 (d=3): ───[H+]───O───
                   │
1 (d=3): ──────────X───

The state is:
 0.58|00⟩ + 0.58|11⟩ + 0.58|22⟩


In [54]:
#Alice operations
aliceOperation(values, q0, circuit)
seeState(circuit)

Circuit: 
0 (d=3): ───[H+]───O───[-1]───[-1]───Z───Z───
                   │
1 (d=3): ──────────X─────────────────────────

The state is:
 0.58|02⟩ + (-0.29-0.5j)|10⟩ + (-0.29+0.5j)|21⟩


In [55]:
#Bob operations
bobOperation(q0, q1, circuit)
seeState(circuit)

Circuit: 
0 (d=3): ───[H+]───O───[-1]───[-1]───Z───Z───O───[H-]───
                   │                         │
1 (d=3): ──────────X─────────────────────────Y──────────

The state is:
 |22⟩
